# MODEL DEVELOPED TO BUILD SRCNN & DNCNN NEURAL NETWORK

In [12]:
#!pip install tensorflow-gpu
#!pip install keras
from keras.models import Sequential,  Model
from keras.layers import Convolution2D,Input,BatchNormalization,Conv2D,Activation,Lambda,Subtract,Conv2DTranspose, PReLU
from keras.regularizers import l2
from keras.layers import  Reshape,Dense,Flatten
# from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from scipy.io import loadmat
import keras.backend as K
# from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
import numpy as np
import math
from scipy import interpolate
#from scipy.misc import imresize


def psnr(target, ref):
    # assume RGB image
    target_data = np.array(target, dtype=float)
    ref_data = np.array(ref, dtype=float)

    diff = target_data - ref_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return rmse #20 * math.log10(255. / rmse)

def interpolation(noisy , SNR , Number_of_pilot , interp):
    noisy_image = np.zeros((40000,72,14,2))

    noisy_image[:,:,:,0] = np.real(noisy)
    noisy_image[:,:,:,1] = np.imag(noisy)


    if (Number_of_pilot == 48):
        idx = [14*i for i in range(1, 72,6)]+[4+14*(i) for i in range(4, 72,6)]+[7+14*(i) for i in range(1, 72,6)]+[11+14*(i) for i in range(4, 72,6)]
    elif (Number_of_pilot == 16):
        idx= [4+14*(i) for i in range(1, 72,9)]+[9+14*(i) for i in range(4, 72,9)]
    elif (Number_of_pilot == 24):
        idx = [14*i for i in range(1,72,9)]+ [6+14*i for i in range(4,72,9)]+ [11+14*i for i in range(1,72,9)]
    elif (Number_of_pilot == 8):
      idx = [4+14*(i) for  i in range(5,72,18)]+[9+14*(i) for i in range(8,72,18)]
    elif (Number_of_pilot == 36):
      idx = [14*(i) for  i in range(1,72,6)]+[6+14*(i) for i in range(4,72,6)] + [11+14*i for i in range(1,72,6)]



    r = [x//14 for x in idx]
    c = [x%14 for x in idx]



    interp_noisy = np.zeros((40000,72,14,2))

    for i in range(len(noisy)):
        z = [noisy_image[i,j,k,0] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,0] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,0] = z_intp
        z = [noisy_image[i,j,k,1] for j,k in zip(r,c)]
        if(interp == 'rbf'):
            f = interpolate.Rbf(np.array(r).astype(float), np.array(c).astype(float), z,function='gaussian')
            X , Y = np.meshgrid(range(72),range(14))
            z_intp = f(X, Y)
            interp_noisy[i,:,:,1] = z_intp.T
        elif(interp == 'spline'):
            tck = interpolate.bisplrep(np.array(r).astype(float), np.array(c).astype(float), z)
            z_intp = interpolate.bisplev(range(72),range(14),tck)
            interp_noisy[i,:,:,1] = z_intp


    interp_noisy = np.concatenate((interp_noisy[:,:,:,0], interp_noisy[:,:,:,1]), axis=0).reshape(80000, 72, 14, 1)
   
    
    return interp_noisy

def SRCNN_model():

    input_shape = (72,14,1)
    x = Input(shape = input_shape)
    c1 = Convolution2D( 64 , (9 , 9) , activation = 'relu', kernel_initializer = 'he_normal', padding='same')(x)
    c2 = Convolution2D( 32 , (1 , 1) , activation = 'relu', kernel_initializer = 'he_normal', padding='same')(c1)
    c3 = Convolution2D( 1 , (5 , 5) , kernel_initializer = 'he_normal', padding='same')(c2)
    #c4 = Input(shape = input_shape)(c3)
    model = Model(inputs = x, outputs = c3)
    ##compile
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) 
    model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error']) 
    return model
    
def SRCNN_train1(train_data ,train_label, val_data , val_label , channel_model , num_pilots , SNR ):
    srcnn_model = SRCNN_model()
    print(srcnn_model.summary())
    
    checkpoint = ModelCheckpoint("SRCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min')
    callbacks_list = [checkpoint]

    srcnn_model.fit(train_data, train_label, batch_size=128, validation_data=(val_data, val_label),
                    callbacks=callbacks_list, shuffle=True, epochs= 300 , verbose=0)
    
    #srcnn_model.save_weights("drive/codes/my_srcnn/SRCNN_SUI5_weights/SRCNN_48_12.h5")
    srcnn_model.save_weights("SRCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")


def SRCNN_train2(train_data ,train_label, val_data , val_label , channel_model , num_pilots , SNR ):
    srcnn_model = SRCNN_model()
    srcnn_model.load_weights("SRCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(12) + ".h5")
    print(srcnn_model.summary())
    
    checkpoint = ModelCheckpoint("SRCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min')
    callbacks_list = [checkpoint]

    srcnn_model.fit(train_data, train_label, batch_size=128, validation_data=(val_data, val_label),
                    callbacks=callbacks_list, shuffle=True, epochs= 300 , verbose=0)
    
    #srcnn_model.save_weights("drive/codes/my_srcnn/SRCNN_SUI5_weights/SRCNN_48_12.h5")
    srcnn_model.save_weights("SRCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
   


def SRCNN_predict(input_data , channel_model , num_pilots , SNR):
    srcnn_model = SRCNN_model()
    srcnn_model.load_weights("SRCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
    predicted  = srcnn_model.predict(input_data)
    return predicted

def SRCNN_predict_output(input_data , channel_model , num_pilots , SNR):
    srcnn_model = SRCNN_model()
    srcnn_model.load_weights("SRCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(22) + ".h5")
    predicted  = srcnn_model.predict(input_data)
    return predicted

  
def DNCNN_model ():
  
    inpt = Input(shape=(None,None,1))
    # 1st layer, Conv+relu
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
    x = Activation('relu')(x)
    # 18 layers, Conv+BN+relu
    for i in range(18):
        x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
        x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
        x = Activation('relu')(x)   
    # last layer, Conv
    x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = Subtract()([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) 
    model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])    
    return model

def DNCNN_train1(train_data ,train_label, val_data , val_label, channel_model , num_pilots , SNR ):
  
  dncnn_model = DNCNN_model()
  print(dncnn_model.summary())

  checkpoint = ModelCheckpoint("DNCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                               save_weights_only=False, mode='min')
  callbacks_list = [checkpoint]

  dncnn_model.fit(train_data, train_label, batch_size=128, validation_data=(val_data, val_label),
                  callbacks=callbacks_list, shuffle=True, epochs= 200 , verbose=0)
  dncnn_model.save_weights("DNCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
  
def DNCNN_train2(train_data ,train_label, val_data , val_label, channel_model , num_pilots , SNR ):
  
  dncnn_model = DNCNN_model()
  dncnn_model.load_weights("DNCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(12) + ".h5")
  print(dncnn_model.summary())

  checkpoint = ModelCheckpoint("DNCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
                               save_weights_only=False, mode='min')
  callbacks_list = [checkpoint]

  dncnn_model.fit(train_data, train_label, batch_size=128, validation_data=(val_data, val_label),
                  callbacks=callbacks_list, shuffle=True, epochs= 200 , verbose=0)
  dncnn_model.save_weights("DNCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(SNR) + ".h5")
  
  
def DNCNN_predict_output(input_data, channel_model , num_pilots , SNR):
  dncnn_model = DNCNN_model()
  dncnn_model.load_weights("DNCNN_" + channel_model +"_"+ str(num_pilots) + "_"  + str(22) + ".h5")
  predicted  = dncnn_model.predict(input_data)
  return predicted

  


# TRAINING THE SRCNN AND DNCNN NEURAL NETWORK 

LOADING PERFECT CHANNEL AND NOISY CHANNEL DATA FOR SNR 12 & 22

In [13]:
import numpy
import math
import pickle
#from scipy.misc import imresize
from scipy.io import loadmat
import matplotlib.pyplot as plt



if __name__ == "__main__":
    # load datasets 
    channel_model = "VehA"
    SNR1 = 12
    SNR2 = 22
    Number_of_pilots = 48
    perfect = loadmat("Perfect_"+ channel_model+'.mat')['My_perfect_H']
    noisy_input1 = loadmat("Noisy_" + channel_model + "_" + "SNR_" + str(SNR1) + ".mat")['My_noisy_H']             
    noisy_input2 = loadmat("Noisy_" + channel_model + "_" + "SNR_" + str(SNR2) + ".mat")['My_noisy_H']
    interp_noisy1 = interpolation(noisy_input1 , SNR1 , Number_of_pilots , 'rbf')
    interp_noisy2 = interpolation(noisy_input2 , SNR2 , Number_of_pilots , 'rbf')
    #pickle.dump(interp_noisy1,open("interp_noisy1.dat","wb"))
    #pickle.dump(interp_noisy2,open("interp_noisy2.dat","wb"))
    #interp_noisy1= pickle.load(open("interp_noisy1.dat","rb"))
    #interp_noisy2= pickle.load(open("interp_noisy2.dat","rb"))
    
    print("\ninterpolation process done\n")

    perfect_image = numpy.zeros((len(perfect),72,14,2))
    perfect_image[:,:,:,0] = numpy.real(perfect)
    perfect_image[:,:,:,1] = numpy.imag(perfect)
    perfect_image = numpy.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)
    
    
    ####### ------ training data ------ #######
       
    idx_random = len(perfect_image) - 8000 # uses 32000 from 36000 as training and the rest as validation
    train_data1, train_label1 = interp_noisy1[:idx_random+1,:,:,:] , perfect_image[:idx_random+1,:,:,:]
    val_data1, val_label1 = interp_noisy1[idx_random+1:idx_random+4001,:,:,:] , perfect_image[idx_random+1:idx_random+4001,:,:,:]
    
    train_data2, train_label2 = interp_noisy2[:idx_random+1,:,:,:] , perfect_image[:idx_random+1,:,:,:]
    val_data2, val_label2 = interp_noisy2[idx_random+1:idx_random+4001,:,:,:] , perfect_image[idx_random+1:idx_random+4001,:,:,:]
    
    test_data1, test_label1 = interp_noisy1[idx_random+4001:,:,:,:] , perfect_image[idx_random+4001:,:,:,:]
    test_data2, test_label2 = interp_noisy2[idx_random+4001:,:,:,:] , perfect_image[idx_random+4001:,:,:,:]

    total_data1, totaltest_label1= interp_noisy1, perfect_image
    total_data2, totaltest_label2= interp_noisy2, perfect_image
    


interpolation process done



TRAINING THE SRCNN & DNCNN FOR SNR 12

In [ ]:
    ####### ------ training SRCNN 1 ------ #######
    SRCNN_train1(train_data1 ,train_label1, val_data1 , val_label1 , channel_model , Number_of_pilots , SNR1 )
    
    ####### ------ prediction using SRCNN 1 ------ #######
    srcnn_pred_train1 = SRCNN_predict(train_data1, channel_model , Number_of_pilots , SNR1)
    srcnn_pred_validation1 = SRCNN_predict(val_data1, channel_model , Number_of_pilots , SNR1)
                          
                      
    ####### ------ training DNCNN 1 ------ #######
    DNCNN_train1(srcnn_pred_train1 ,train_label1, srcnn_pred_validation1 , val_label1 , channel_model , Number_of_pilots , SNR1 )
    

TRAINING THE SRCNN & DNCNN FOR SNR 22 (with initializing weights of nn from previous trained  nn for SNR 12)

In [ ]:
   
    ####### ------ training SRCNN 2 ------ #######
    SRCNN_train2(train_data2 ,train_label2, val_data2 , val_label2 , channel_model , Number_of_pilots , SNR2 )
    
    ####### ------ prediction using SRCNN 2 ------ #######
    srcnn_pred_train2 = SRCNN_predict(train_data2, channel_model , Number_of_pilots , SNR2)
    srcnn_pred_validation2 = SRCNN_predict(val_data2, channel_model , Number_of_pilots , SNR2)
    
    ####### ------ training DNCNN 2 ------ #######
    DNCNN_train2(srcnn_pred_train2 ,train_label2, srcnn_pred_validation2 , val_label2 , channel_model , Number_of_pilots , SNR2 )
   
    #-----------------------------------------------------------------------------------------------------------------------------#                    


TESTING THE TRAINED NN FOR SNR 12 & 22

In [15]:
    #-------------testing whole 2 NN------------#
    srcnn_test1 = SRCNN_predict_output(test_data1, channel_model , Number_of_pilots , SNR1)
    dncnn_test1 = DNCNN_predict_output(srcnn_test1, channel_model , Number_of_pilots , SNR1)
    
    errorfound1= psnr(test_label1,dncnn_test1)
    print("errorfound1",errorfound1)
    
    srcnn_test2 = SRCNN_predict_output(test_data2, channel_model , Number_of_pilots , SNR2)
    dncnn_test2 = DNCNN_predict_output(srcnn_test2, channel_model , Number_of_pilots , SNR2)
    
    errorfound2= psnr(test_label2,dncnn_test2)
    print("errorfound2",errorfound2)

    srcnn_test3 = SRCNN_predict_output(total_data1, channel_model , Number_of_pilots , SNR2)
    dncnn_test3 = DNCNN_predict_output(srcnn_test3, channel_model , Number_of_pilots , SNR2)
    
    errorfound3= psnr(totaltest_label1,dncnn_test3)
    print("errorfound3",errorfound3)

    srcnn_test4 = SRCNN_predict_output(total_data2, channel_model , Number_of_pilots , SNR2)
    dncnn_test4 = DNCNN_predict_output(srcnn_test4, channel_model , Number_of_pilots , SNR2)
    
    errorfound4 = psnr(totaltest_label2,dncnn_test4)
    print("errorfound4",errorfound4)


errorfound1 0.09067199198952629
errorfound2 0.031095590587197998
errorfound3 0.09053034615361853
errorfound4 0.030869711687326738


# TESTING THE NN FOR DIFFERNENT SNR 

In [ ]:
import numpy
import math
import pickle
from scipy.io import loadmat
import matplotlib.pyplot as plt

if __name__ == "__main__":
    # load datasets 
    channel_model = "VehA"
    SNR = int(input("enter the SNR"))
    Number_of_pilots = 48
    perfect = loadmat("Perfect_"+ channel_model+'.mat')['My_perfect_H']
    noisy_input = loadmat("Noisy_" + channel_model + "_" + "SNR_" + str(SNR) + ".mat")['My_noisy_H']             
    interp_noisy = interpolation(noisy_input , SNR , Number_of_pilots , 'rbf')
    print("\ninterpolation process done\n")
    perfect_image = numpy.zeros((len(perfect),72,14,2))
    perfect_image[:,:,:,0] = numpy.real(perfect)
    perfect_image[:,:,:,1] = numpy.imag(perfect)
    perfect_image = numpy.concatenate((perfect_image[:,:,:,0], perfect_image[:,:,:,1]), axis=0).reshape(2*len(perfect), 72, 14, 1)  
    
    input_data, input_label= interp_noisy, perfect_image
    srcnn_test = SRCNN_predict_output(input_data, channel_model , Number_of_pilots , SNR)
    dncnn_test = DNCNN_predict_output(srcnn_test, channel_model , Number_of_pilots , SNR)
    
    errorfound= psnr(input_label,dncnn_test)
    print("RMSE",errorfound)

# CODE TO KILL SESSION AND FREE RAM

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)